In [1]:
import os

In [2]:
from src.dataprep.TracingChecker import TracingChecker

In [3]:
debug = True

In [4]:
# DEPRECATED WITH NEW SAMPLING APPROACH
# existing_labeled_dataset = r".\data\tracings\invalid_rat301_test"
# TracingChecker(existing_labeled_dataset).check()

In [5]:
read_path = r".\data\project_scans"
subregion_list = ['contra_inner', 'contra_outer', 'ipsi_inner', 'ipsi_outer']
# How should I explain and insert groups here?
# this is just an example 
groups = [['ipsi_outer', 'contra_outer'], ['ipsi_inner', 'contra_inner']]
group_labels = ["Superficial", "Deep Layer"]

train_write_path = r".\data\tracings_round_2\train" 
test_write_path = r".\data\tracings_round_2\test"

os.makedirs(train_write_path, exist_ok=True)
os.makedirs(test_write_path, exist_ok=True)

train_sample_dimensions = (256,256) # can be bigger because for training we can accept training data is non i.i.d 
test_sample_dimensions = (256,256) # smaller because of regional dependance, we want to have as big an effective sample size as possible

In [6]:
# sample the structure
uniform_sampling = True
stratify_by_region = True

n_train = 100
n_test = 100
channel = "th"

In [7]:
from src.dataprep.SamplingStrategies import SRS, StratifiedSamplingStrategy

# SRS strategy doesn't need to be aware of groups for training, the models should perform well across images. 
training_sample_strategy = SRS(read_path, channel, train_sample_dimensions, stratify_by_region,
                               stratify_group=True, groups=None)

# Test sampling strategy, however, should be specialized to answer a specific question
test_sample_strategy = StratifiedSamplingStrategy(read_path, channel, test_sample_dimensions, stratify_by_region,
                                stratify_group=True, groups=groups)

TypeError: src.dataprep.SamplingStrategies.SamplingStrategy.__init__() got multiple values for keyword argument 'stratify_group'

In [ ]:
# Some sampling strategies can be used to suggest n_train or n_test
# Integrate this

In [ ]:
from .src.dataprep.SampleSaver import SampleSaver

train_samp = SampleSaver(train_write_path, training_sample_strategy)
if not debug: train_samp.create_dataset(n_train)

test_samp = SampleSaver(test_write_path, test_sample_strategy)
if not debug: test_samp.create_dataset(n_test)

In [ ]:
# If the project necessitates a validation dataset (lots of hyperparameter tuning and large enough labeling budget)
# it can be done, but in my case I don't see a reason
use_validation = False
if use_validation:
    # The sampling strategy of that set also depends on what it would be used for
    val_write_path = r".\data\tracings_round_2\val"
    os.makedirs(val_write_path, exist_ok=True)
    val_sample_strategy = SRS(read_path, channel, test_sample_dimensions, stratify_by_region,
                               stratify_group=True, groups=None)
    val_samp = SampleSaver(test_write_path, test_sample_strategy)
    if not debug: val_samp.create_dataset(n_test)

In [ ]:
if not debug: # Datasets should have been created
    train_checker = TracingChecker(root_read_dir=train_write_path)
    train_checker.check()

    test_checker = TracingChecker(root_read_dir=test_write_path)
    test_checker.check()
    
    if use_validation: 
        val_checker = TracingChecker(root_read_dir=val_write_path)
        val_checker.check()
    # If all of this looks good, you can proceed to the fun part... manually tracing axons :) ! 

In [ ]:
# Further tests as needed

"""complete_dataset = train_write_path
train_checker = TracingChecker(root_read_dir=complete_dataset)
train_checker.check()


for region in subregion_list:
    paths = train_checker.get_img_paths_from_region_group([region])
    print(len(paths))
    
paths = train_checker.get_all_original_folders()
from collections import Counter

print(Counter(paths))"""